In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, ShuffleSplit, GridSearchCV, cross_val_score, cross_validate
import os
import glob
from scipy.stats import norm
import math
import random
from rdkit import Chem
from sklearn import preprocessing  

T_0 = pd.read_csv('./Data/LGAC-label.csv')
T=np.array(T_0)
X=pd.read_csv('./Data/LGAC-morgan.csv')
X=np.array(X)

# 1. LightGBM

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train,  X_test,  T_train,  T_test  =  train_test_split(X,T,test_size=0.20,random_state=34)
plt.style.use('ggplot')
regressor = lgb.LGBMRegressor(n_estimators=2000)
regressor.fit(X_train,T_train)
lgb.LGBMRegressor(boosting_type='gbdt', class_weight=0.5, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31,random_state=34, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

y_true_test=T_test
y_pred_test=regressor.predict(X_test)

print("y_true_test shape:", y_true_test.shape)
print("y_pred_test shape:", y_pred_test.shape)

y_true_test = np.ravel(y_true_test)
y_pred_test = np.ravel(y_pred_test)
y_true=T_test
print ("test set r^2 score",r2_score(T_test,y_pred_test))
print("test set MAE",mean_absolute_error(y_true, y_pred_test))
print("test set RMSE",np.sqrt(mean_squared_error(y_true,y_pred_test)))

E:\Anaconda\envs\dye37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


y_true_test shape: (1279, 1)
y_pred_test shape: (1279,)
test set r^2 score 0.8491837239385192
test set MAE 0.09243093312792663
test set RMSE 0.1447452749384954


# 2. XGBoost

In [3]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.1, random_state=42)

xg_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    base_score=0.5,
    booster='gbtree',
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    enable_categorical=False,
    gamma=0,
    max_delta_step=0,
    max_depth=6,
    min_child_weight=1,
    missing=np.nan,
    n_estimators=100,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    subsample=1,
    random_state=42
)

xg_reg.fit(X_train, T_train)

train_r2 = xg_reg.score(X_train, T_train)
test_r2 = xg_reg.score(X_test, T_test)

print(f"测试集 R² score: {test_r2:.4f}")

y_pred_train = xg_reg.predict(X_train)
y_pred_test = xg_reg.predict(X_test)

train_mae = mean_absolute_error(T_train, y_pred_train)
test_mae = mean_absolute_error(T_test, y_pred_test)

train_rmse = np.sqrt(mean_squared_error(T_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(T_test, y_pred_test))

print(f"test set MAE: {test_mae:.4f}")
print(f"test set RMSE: {test_rmse:.4f}")


测试集 R² score: 0.8332
test set MAE: 0.1092
test set RMSE: 0.1578


# GBRT

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.1, random_state=42)

gbr_reg = GradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=31,
    max_features=300,
    min_samples_leaf=20,
    n_estimators=200,
    random_state=42
)

gbr_reg.fit(X_train, T_train)

train_r2 = gbr_reg.score(X_train, T_train)
test_r2 = gbr_reg.score(X_test, T_test)
print(f"test set R² score: {test_r2:.3f}")

y_pred_train = gbr_reg.predict(X_train)
y_pred_test = gbr_reg.predict(X_test)

train_mae = mean_absolute_error(T_train, y_pred_train)
test_mae = mean_absolute_error(T_test, y_pred_test)

train_rmse = np.sqrt(mean_squared_error(T_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(T_test, y_pred_test))

print(f"test set MAE: {test_mae:.3f}")
print(f"test set RMSE: {test_rmse:.3f}")

E:\Anaconda\envs\dye37\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test set R² score: 0.868
test set MAE: 0.095
test set RMSE: 0.141
